In [7]:
# this is the updated code for v8.5.0
# this is a code that fit all basins

import pandas as pd
import numpy as np
import datetime
import math
import os
import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline
# print(os.getcwd())
# os.listdir()

In [8]:
# set the working folder before start
%cd "G:\My Drive\Veld_Code\EF\v811\Raw"

# Assign Basin here, which are: WIL, APP, EF, PER, HAY, BAR, DJ, MC
BASIN = 'EF'

# if we use ENERTEL allocated production data or not (for TX and LA)
# ENERTEL = 'NO'







# this is to get distance in feet
# 1 km = 3280.84 ft, the last part return will be km times 3280.84

def get_distance(lat_1, lng_1, lat_2, lng_2): 
    lng_1, lat_1, lng_2, lat_2 = map(math.radians, [lng_1, lat_1, lng_2, lat_2])
    d_lat = lat_2 - lat_1
    d_lng = lng_2 - lng_1 

    temp = (  
         math.sin(d_lat / 2) ** 2 
       + math.cos(lat_1) 
       * math.cos(lat_2) 
       * math.sin(d_lng / 2) ** 2
    )

    return 6373.0 * 3280.84 * (2 * math.atan2(math.sqrt(temp), math.sqrt(1 - temp)))

# get_distance(52.2296756,21.0122287,52.406374,16.9251681)
# should be 278.546 km * 3280.84 ft/km = 913,864.85864 ft


# header_needed_columns = ['API','MeasuredDepth',
#        'TrueVerticalDepth', 'LateralLength', 
#         'BHLatitude', 'BHLongitude','Latitude', 'Longitude','CurrentOperator', 'OriginalOperator',
#         'WellName','WellType', 'WellStatus',
#        'Field', 'County', 'State',
#        'Country', 'WellBoreProfile','PermitDate',
#        'SpudDate', 'CompletionDate', 'Basin', 'Play', 'Lease', 'LeaseId', 'GroundElevation',
#        'PrimaryFormation','ReportedCurrentOperator','ReportedOriginalOperator','FirstProdDate','LastProdDate']

header_needed_columns = ['API','MeasuredDepth',
       'TrueVerticalDepth', 'LateralLength', 'Latitude', 'Longitude','Heel_Lat','Heel_Long',
        'BHLatitude', 'BHLongitude','CurrentOperator', 'OriginalOperator',
        'WellName','WellType', 'WellStatus',
       'Field', 'County', 'State',
       'Country', 'WellBoreProfile','PermitDate',
       'SpudDate', 'CompletionDate', 'Basin', 'Play', 'Lease', 'LeaseId', 'GroundElevation',
       'PrimaryFormation','ReportedCurrentOperator','ReportedOriginalOperator','FirstProdDate', 'LastProdDate']

G:\My Drive\Veld_Code\EF\v811\Raw


### Part I: Data Pre-Processing

In [42]:
# function 0.1: merge the new data with our golden database
# then fiture out what data are still missing

def Merge_w_Previous(df_new, df_golden):
    
    
    print('raw file:')
    print(df_new.shape)
    print('*'*100)
    
    df_new = df_new[header_needed_columns]
    df_new['API_10'] = df_new['API'] / 10000
    df_new['API_10'] = df_new['API_10'].astype(np.int64)
    df_new['API'] = df_new['API_10'] * 10000
    df_new['API'] = df_new['API'].astype(np.int64)
    df_new = df_new[df_new['API'] > 0]
    df_new.drop_duplicates(subset=['API'],keep='last',inplace=True)
    df_new = df_new.reset_index(drop=True)
    
    
    df_match_golden = df_new[df_new['API'].isin(df_golden['API'])]
    df_set_aside = df_new[~df_new['API'].isin(df_golden['API'])]
    
    print('raw data total:')
    print(df_new.shape)
    print('*'*100)
    print('golden data total:')
    print(df_golden.shape)
    print('*'*100)
    print('raw data that matched golden database API:')
    print(df_match_golden.shape)
    print('*'*100)
    print('the well API that are not in golden database and set aside:')
    print(df_set_aside.shape)    
    
    df_match_golden = df_match_golden.drop(['MeasuredDepth','TrueVerticalDepth','LateralLength','Latitude','Longitude',
                                              'Heel_Lat','Heel_Long','BHLatitude','BHLongitude'], axis=1)
    df_match_golden = pd.merge(df_match_golden,df_golden[['API','MeasuredDepth','TrueVerticalDepth','LateralLength',
                                              'Latitude','Longitude','Heel_Lat','Heel_Long','BHLatitude','BHLongitude'
                                             ]],on='API', how='inner')

    df_match_golden = df_match_golden[header_needed_columns]
    
    df_new_merge = pd.concat([df_match_golden,df_set_aside])
    
    df_new_merge = df_new_merge[df_new_merge['MeasuredDepth'] != 'remove']
    
    print('*'*100)
    print('merged raw files with golden database after removing vertical wells etc:')
    print(df_new_merge.shape)      
    
    return df_new_merge
    
    





df_raw_header_00 = pd.read_csv('0000_Raw_Header_Final_Step_0.csv') 
df_golden = pd.read_excel('0000_Golden_Missing_Database_20211130.xlsx')


# Merge_w_Previous(df_raw_header_00, df_golden)

df_raw_header_01 = Merge_w_Previous(df_raw_header_00, df_golden)
df_raw_header_01.to_csv('df_raw_header_v811_merge_w_previous.csv',index=False)



raw file:
(27645, 69)
****************************************************************************************************
raw data total:
(27645, 34)
****************************************************************************************************
golden data total:
(27786, 11)
****************************************************************************************************
raw data that matched golden database API:
(27562, 34)
****************************************************************************************************
the well API that are not in golden database and set aside:
(83, 34)
****************************************************************************************************
merged raw files with golden database after removing vertical wells etc:
(27628, 34)


In [45]:



# function 0.2: just using the Raw_Header to get the missing TVD, LL, MD, SHL, BHL for Rollyn
# this function is for unconventional wells only

def Get_Missing_Data(df_header):
    
    
    # get the distance between SHL and BHL
    for i in range(df_header.shape[0]):
        df_header.loc[i,'SL_BH_Distance'] = get_distance(df_header.loc[i,'Latitude'],
                                                         df_header.loc[i,'Longitude'],
                                                         df_header.loc[i,'BHLatitude'],
                                                         df_header.loc[i,'BHLongitude']
        )
        
    # get the suspicious wells with SHL to BHL distance more than 21120' or less than 500'
    df_header_bad_sh_bh_distance = df_header[ (df_header['SL_BH_Distance'] > 21120) 
#                                              |
#                                               (df_header['SL_BH_Distance'] < 10)
                                            ] 
        
        
        
    # get missing TVD
    df_header_tvd = df_header[df_header['TrueVerticalDepth'] > 0]
    df_header_tvd = df_header_tvd[df_header_tvd['MeasuredDepth'] != df_header_tvd['TrueVerticalDepth']]
    df_header_no_TVD = df_header[~df_header.API.isin(df_header_tvd.API)]
    
    # get missing LL
    df_header_LL = df_header[df_header['LateralLength'] > 1000]
    df_header_LL = df_header_LL[df_header_LL['LateralLength'] < 25000]
    df_header_no_LL = df_header[~df_header.API.isin(df_header_LL.API)]
    
    # get missing operator
    df_header_no_Operator = df_header[df_header['CurrentOperator'].isnull()]
    
    # get the missing SHL
    df_header_no_SHL = df_header[df_header['Latitude'].isnull()]
    
    # get the missing BHL
    df_header_no_BHL = df_header[df_header['BHLatitude'].isnull()]
    
    
    df_missing_well_data = pd.concat([df_header_no_TVD,
                                      df_header_no_LL,
                                      df_header_no_Operator,
                                      df_header_bad_sh_bh_distance,
                                      df_header_no_SHL,
                                      df_header_no_BHL
                                     ])
    
    # drop the duplicates for Rollyn
    df_missing_well_data.drop_duplicates(subset='API', inplace = True)
    
    
    return df_missing_well_data


Raw_Header_0 = pd.read_csv('df_raw_header_v811_merge_w_previous.csv')



Raw_Header_0 = Raw_Header_0[header_needed_columns]
df_missing_well_data = Get_Missing_Data(Raw_Header_0)
df_missing_well_data.to_excel("df_missing_well_data.xlsx",index=False)
df_missing_well_data.shape


(14, 34)

In [3]:
# function 1: merge 2 seperate data if needed, and re-export as raw data standard
# use case is to add the private data to exisiting public database

Raw_Header_1 = pd.read_csv('df_raw_header_v811_merge_w_previous.csv')
Raw_Header_2 = pd.read_csv('EF_v811_Addon_20220305_22_Wells-20220932050-dR0C6-Header.csv')
Raw_Header_2 = Raw_Header_2[header_needed_columns]

Raw_Monthly_Production_1 = pd.read_csv('EF_v811_Base_20220305-20220632031-1BD8P-Production.csv')
Raw_Monthly_Production_2 = pd.read_csv('EF_v811_Addon_20220305_22_Wells-20220932050-dR0C6-Production.csv')

# Raw_Completion_1 = pd.read_csv('11.29.2021_Permian_30239_Wells-202129111735-Hw2YK-Stimulation.csv')
# Raw_Completion_2 = pd.read_csv('11.29.2021_Permian_3328_Permits_2Yr-202129111722-6g04r-Stimulation.csv')


Raw_Header = pd.concat([Raw_Header_1,Raw_Header_2])
Raw_Monthly_Production = pd.concat([Raw_Monthly_Production_1,Raw_Monthly_Production_2])
# Raw_Completion = pd.concat([Raw_Completion_1,Raw_Completion_2])

Raw_Header.to_csv('0000_Raw_Header_Final.csv',index=False)
Raw_Monthly_Production.to_csv('0000_Raw_Monthly_Production.csv',index=False)
# Raw_Completion.to_csv('0000_Raw_Completion.csv')


print(Raw_Header_1.shape)
print(Raw_Header_1.API.nunique())
print(Raw_Header_2.shape)
print(Raw_Header_2.API.nunique())
print(Raw_Header.shape)
print(Raw_Header.API.nunique())


print("****************")

print(Raw_Monthly_Production_1.shape)
print(Raw_Monthly_Production_1.API.nunique())
print(Raw_Monthly_Production_2.shape)
print(Raw_Monthly_Production_2.API.nunique())
print(Raw_Monthly_Production.shape)
print(Raw_Monthly_Production.API.nunique())


print("****************")

# print(Raw_Completion_1.shape)
# print(Raw_Completion_1.API.nunique())
# print(Raw_Completion_2.shape)
# print(Raw_Completion_2.API.nunique())
# print(Raw_Completion.shape)
# print(Raw_Completion.API.nunique())



(27619, 34)
27619
(22, 33)
22
(27641, 34)
27641
****************
(2230817, 28)
27626
(274, 28)
7
(2231091, 28)
27633
****************


In [9]:
# Before running start, rename all input files to:
# "0000_Raw_Header.csv"
# "0000_Raw_Monthly_Production.csv"
# "0000_Raw_Completion.csv"
# "0000_Raw_Spacing.csv"

# Import raw header, completion, production summary and monthly production data from WellDatabase
Raw_Header                =    pd.read_csv('0000_Raw_Header_Final.csv')
Raw_Monthly_Production    =    pd.read_csv('0000_Raw_Monthly_Production.csv')
Raw_Completion            =    pd.read_csv('0000_Raw_Completion.csv')
# Raw_Spacing               =    pd.read_csv('0000_Raw_Spacing.csv')


Raw_Header['API_10'] = Raw_Header['API'] / 10000
Raw_Header['API_10'] = Raw_Header['API_10'].astype(np.int64)
Raw_Header['API'] = Raw_Header['API_10'] * 10000
Raw_Header['API'] = Raw_Header['API'].astype(np.int64)
Raw_Header.drop_duplicates(subset=['API'],keep='last',inplace=True)
Raw_Header = Raw_Header.reset_index(drop=True)

# export the raw header for future use 
# mainly to generate the disqualified list and pie chart, otherwise it will cause bug
Raw_Header.to_csv('0000_Raw_Header_Final.csv',index=False)


Raw_Monthly_Production.dropna(subset=['API'],inplace=True)
Raw_Monthly_Production['API'] = Raw_Monthly_Production['API'].astype(np.int64)
Raw_Completion.dropna(subset=['API'],inplace=True)
Raw_Completion['API'] = Raw_Completion['API'].astype(np.int64)


# use the filtered column
Raw_Header = Raw_Header[header_needed_columns]

# fill all the Primary Formation and Baisn Column
Raw_Header['PrimaryFormation'].fillna('UNKNOWN', inplace=True)

if BASIN == 'WIL':
    Raw_Header['Basin'] = 'WILLISTON'

if BASIN == 'APP':
    Raw_Header['Basin'] = 'APPALACHIAN'
    
if BASIN == 'EF':
    Raw_Header['Basin'] = 'EAGLE FORD'

if BASIN == 'HAY':
    Raw_Header['Basin'] = 'HAYNESVILLE'

if BASIN == 'DJ':
    Raw_Header['Basin'] = 'DJ'
    
if BASIN == 'BAR':
    Raw_Header['Basin'] = 'BARNETT'

if BASIN == 'PER':
    Raw_Header['Basin'] = 'PERMIAN'

if BASIN == 'MC':
    Raw_Header['Basin'] = 'MIDCONTINENT'

        

# fix the edge cases where SHL is null and BHL has values
Raw_Header.Latitude = np.where(Raw_Header.Latitude.isnull(), Raw_Header.BHLatitude, Raw_Header.Latitude)
Raw_Header.Longitude = np.where(Raw_Header.Longitude.isnull(), Raw_Header.BHLongitude, Raw_Header.Longitude)

# fix the Current Operator blanks automatically 
Raw_Header.CurrentOperator = np.where(Raw_Header.CurrentOperator.isnull(), 
                                      Raw_Header.OriginalOperator, Raw_Header.CurrentOperator)


In [10]:
print(Raw_Header.shape)
print(Raw_Header.API.nunique())
print('*'*20)
print(Raw_Monthly_Production.shape)
print(Raw_Monthly_Production.API.nunique())
print('*'*20)
print(Raw_Completion.shape)
print(Raw_Completion.API.nunique())
# print(Raw_Spacing.shape)
# print(Raw_Spacing.API.nunique())

(27641, 33)
27641
********************
(2576773, 10)
27634
********************
(39767, 62)
24529


In [11]:
# function 2: check the API of all header, production, completion and spacing
# if the API do not agree, throw error
# all API length should all be 14 digits and end with 0000
# also generate API_10 in this stage and make all API and API_10 int64 format 
print('before')
print("min API in header           = ", len(str(np.min(Raw_Header.API))))
print("min API in QMM              = ", len(str(np.min(Raw_Monthly_Production.API))))
print("min API in Completion       = ", len(str(np.min(Raw_Completion.API))))
# print("min API in Spacing          = ", len(str(np.min(Raw_Spacing.API))))
print('*'*20)
API_Digit_Difference = len(str(np.min(Raw_Header.API))) - len(str(np.min(Raw_Monthly_Production.API)))
API_Digit_Difference_comp = len(str(np.min(Raw_Header.API))) - len(str(np.min(Raw_Completion.API)))


# set_aside the small API and change that to 14 digits
# this is assume there is only one API alternatives
# then the max API length is not equal to min API length anymore
# should thorw an error to prevent going forward

# if there are 2 cases, where you have a mix of 10 digit and 12 digit API mixed, then
# I need to manually do this job

def Fill_14_Digits_API(table):
    
    # get the API < 14 digits
    # 10 **13 is the minimal of 14 digit API
    table_less = table[table.API < 10**13] 
    
    # set aside the good API
    table_set_aside = table[~table.API.isin(table_less.API)]
    
    # fill the less API with 0 towards 14 digit API
    table_less['API'] = table_less['API']*(10**API_Digit_Difference)

    # merge (concat) the less and set_aside
    table_new = pd.concat([table_less,table_set_aside])
    
    return table_new

def Fill_14_Digits_API_completion(table):
    
    # get the API < 14 digits
    # 10 **13 is the minimal of 14 digit API
    table_less = table[table.API < 10**13] 
    
    # set aside the good API
    table_set_aside = table[~table.API.isin(table_less.API)]
    
    # fill the less API with 0 towards 14 digit API
    table_less['API'] = table_less['API']*(10**API_Digit_Difference_comp)

    # merge (concat) the less and set_aside
    table_new = pd.concat([table_less,table_set_aside])
    
    return table_new





# if BASIN != 'PER':
Raw_Header = Fill_14_Digits_API(Raw_Header)
Raw_Monthly_Production = Fill_14_Digits_API(Raw_Monthly_Production)
Raw_Completion = Fill_14_Digits_API_completion(Raw_Completion)
    # Raw_Spacing = Fill_14_Digits_API(Raw_Spacing)
    

# make sure all API are end with 0000
Raw_Monthly_Production['API_10'] = Raw_Monthly_Production['API'] / 10000
Raw_Monthly_Production['API_10'] = Raw_Monthly_Production['API_10'].astype(np.int64)
Raw_Monthly_Production['API'] = Raw_Monthly_Production['API_10'] * 10000
Raw_Monthly_Production['API'] = Raw_Monthly_Production['API'].astype(np.int64)

Raw_Completion['API_10'] = Raw_Completion['API'] / 10000
Raw_Completion['API_10'] = Raw_Completion['API_10'].astype(np.int64)
Raw_Completion['API'] = Raw_Completion['API_10'] * 10000
Raw_Completion['API'] = Raw_Completion['API'].astype(np.int64)
    



print('after, check min and max, they have to be equal in order to move forward')
print("min API in header           = ", len(str(np.min(Raw_Header.API)))   )
print("min API in QMM              = ", len(str(np.min(Raw_Monthly_Production.API)))   )
print("min API in Completion       = ", len(str(np.min(Raw_Completion.API)))   )
# print("min API in Spacing          = ", len(str(np.min(Raw_Spacing.API)))   )

print('*'*20)
print("max API in header           = ", len(str(np.max(Raw_Header.API))))
print("max API in QMM              = ", len(str(np.max(Raw_Monthly_Production.API)))   )
print("max API in Completion       = ", len(str(np.max(Raw_Completion.API)))   )
# print("max API in Spacing          = ", len(str(np.max(Raw_Spacing.API))))

if len(str(np.min(Raw_Monthly_Production.API))) != len(str(np.max(Raw_Monthly_Production.API))):
    raise ValueError('max and min API digits not equal, go check')

if len(str(np.min(Raw_Monthly_Production.API))) != 14:
    raise ValueError('min API digits not equal tp 14, go check')
    
if len(str(np.max(Raw_Monthly_Production.API))) != 14:
    raise ValueError('min API digits not equal tp 14, go check')


before
min API in header           =  14
min API in QMM              =  14
min API in Completion       =  10
********************
after, check min and max, they have to be equal in order to move forward
min API in header           =  14
min API in QMM              =  14
min API in Completion       =  14
********************
max API in header           =  14
max API in QMM              =  14
max API in Completion       =  14


In [12]:
# function 3.1: calculate spacing in house (not use WellDB spacing anymore)

def Get_Spacing(df_header):
    
    well_count = df_header.shape[0]
    
    # initialize the lists to be used
    mid_latitude, mid_longitude = [], []
    horizontal_spacing, total_spacing = [], []

    # Calculate Mid-lateral Latitude and Longitude
    for i in range(well_count):
        mid_lat = (df_header.loc[i,'Latitude'] + df_header.loc[i,'BHLatitude']) / 2
        mid_long = (df_header.loc[i,'Longitude'] + df_header.loc[i,'BHLongitude']) / 2
        mid_latitude.append(mid_lat)
        mid_longitude.append(mid_long)        

    df_header['mid_latitude'] = mid_latitude
    df_header['mid_longitude'] = mid_longitude

    # now need to group the wells into 6 miles radius inside a loop
    # target well i has an offset wells within 6 miles, which is 0.1 in lat and long

    for i in range(well_count):
        
        target_mid_lat = df_header.loc[i,'mid_latitude']
        target_mid_long = df_header.loc[i,'mid_longitude']

        # first, get a group of wells within 0.1 lat/long to target well
        df_offset = df_header[ (df_header['mid_latitude'] <=  target_mid_lat + 0.1) &
                               (df_header['mid_latitude'] >=  target_mid_lat - 0.1) &
                               (df_header['mid_longitude'] <=  target_mid_long + 0.1) &
                               (df_header['mid_longitude'] >=  target_mid_long - 0.1) 
                             ]

        df_offset = df_offset.reset_index(drop=True)
        
        # set the maximum cap of horizontal spacing to be 5280
        min_distance = 5280
        total_distance = 5280
        
        # calculate the distance between mid point to the offset wells
        for j in range(df_offset.shape[0]):
            distance_to_target = get_distance(
                target_mid_lat,
                target_mid_long,
                df_offset.loc[j,'mid_latitude'],
                df_offset.loc[j,'mid_longitude'],        
            )   
        
        # if distanc_to_target is less than 5280, and it is > 0, so update the min distance
        # min_distance should be the horizontal spacing
            if (distance_to_target < min_distance) & (distance_to_target > 0):
                min_distance = distance_to_target
                
                # get the horizontal distance then get the total distance 
                # adding vertical portion
                
                # fillna of TVD
                df_header['TrueVerticalDepth'].fillna((df_header['TrueVerticalDepth'].mean()), inplace=True)
                
                # get the total distance using horizontal and vertical distance
                vertical_difference = df_header.loc[i,'TrueVerticalDepth'] - df_header.loc[j,'TrueVerticalDepth']
                total_distance = np.sqrt(vertical_difference**2 + min_distance**2)
                
        min_distance = int(min_distance)
#         print(min_distance)       
        total_distance = int(min(total_distance,5280))
        
        horizontal_spacing.append(min_distance)
        total_spacing.append(total_distance)
    
    # finish and output
    df_header['Spacing'] = horizontal_spacing
    df_header['total_spacing'] = total_spacing
    
    df_header['Spacing'][df_header['Spacing'] == 0] = 100
    df_header['total_spacing'][df_header['total_spacing'] == 0] = 100
    
    return df_header
        
Raw_Header = Get_Spacing(Raw_Header)

In [13]:
# function 3.2: merge completion and spacing to the raw header



# only 1 line of code to get the completion data
# Extract total proppant column from completion table
df_completion = Raw_Completion[['API','TotalProppantMass']]
df_completion = df_completion.sort_values(by=['API', 'TotalProppantMass'])
df_completion = df_completion.drop_duplicates(subset='API', keep="last")
df_completion = df_completion[df_completion.API.isin(Raw_Header.API)]
# df_completion

# Sort the header file


Raw_Header = Raw_Header[Raw_Header['API'] > 0]
Raw_Header = Raw_Header.sort_values(['API'], ascending = True)
Raw_Header.reset_index(drop = True,inplace=True)





# Merge completion table and header table, with left join
df_header = pd.merge(Raw_Header, df_completion, how='left', on=['API'])
df_header = df_header.drop_duplicates()

# get the frac inteensity
df_header['FracIntensity'] = df_header['TotalProppantMass'] / df_header['LateralLength']

# fill the frac intensity of inf and na with 0
df_header['FracIntensity'] = df_header['FracIntensity'].replace(-np.inf, np.nan)           
df_header['FracIntensity'].fillna(0,inplace=True)

# limit the frac intensity to be <= 10000
df_header['FracIntensity'][df_header['FracIntensity'] > 10000] = 10000


In [14]:
# function 3.3: fill all the blank lease ID with lease numbers
# make sure all lease ID has values 
# this is for Texas and Luisianna only 

if BASIN in ['EF', 'PER', 'HAY']:
#     df_header = Fill_LeaseID(df_header)
    df_header.LeaseId = np.where(df_header.LeaseId.isnull(), df_header.Lease, df_header.LeaseId)
            
# df_header.LeaseId

In [15]:
# for Permian only

if BASIN in ['PER']:
    
    BS_List = ['ABO', 'ABO NORTH', 'BONE SPRING', 'BONE SPRING, SOUTHWEST',
       'BONE SPRING, SOUTHWWEST', 'DELAWARE', 'DELAWARE/BS',
       'FORTY NINER RIDGE BONE SPRING WEST', 'GLORIETA',
       'LOWER BONE SPRING', 'LOWER BS', 'LWR BONE SPRIN', 'SA-YESO',
           'UPPER BONE SPR', 'UPPER BONE SPRING SHALE','FORTY NINER RIDGE BS WEST','UPPER BS SHALE','UPPER BSS',
       'UPPER BONE SPRINGS', 'UPR BONE SPRIN',  'YESO', 'BS'
          ]
           
           
    SP_List = [ 'SPRABERRY', 'SP'        ]
    WC_List = [ 'WLFCMPPENN CONS',
           'WOLFBONE', 'WOLFCAMP', 'WOLFELLEN CONS','DEVONIAN','CONSOLIDATED','WC'       ]
    
    
    for index, row in df_header.iterrows():
    
        if df_header.loc[index,'PrimaryFormation'] in BS_List:
            df_header.loc[index,'PrimaryFormation'] = 'BS'

        if df_header.loc[index,'PrimaryFormation'] in SP_List:
            df_header.loc[index,'PrimaryFormation'] = 'SP'      

        if df_header.loc[index,'PrimaryFormation'] in WC_List:
            df_header.loc[index,'PrimaryFormation'] = 'WC'
    
    print(df_header.PrimaryFormation.unique())
    
    
    
    
    
    
    
    

In [16]:
# Raw_Monthly_Production['WellWater'] = 0
# Raw_Monthly_Production['TotalWater'] = 0
# Raw_Monthly_Production['TotalGas'] = 0
# Raw_Monthly_Production['MonthNumber'] = 0
# Raw_Monthly_Production['TotalOil'] = 0






In [17]:
# function 4.1: remember to assign all production days to 30.4375

Raw_Monthly_Production['Days'] = 30.4375

# function 4.2: initialize Raw Monthly Production Data

QMM_needed_columns = ['API','ReportDate','Days','TotalOil','WellOil',
                     'TotalGas','WellGas','TotalWater','WellWater','MonthNumber']

df_raw_production = Raw_Monthly_Production[QMM_needed_columns]
df_raw_production['ReportDate'] = pd.to_datetime(df_raw_production['ReportDate'])

# make sure the headers and raw productions are matching
df_raw_production = df_raw_production[df_raw_production.API.isin(df_header.API)] 
df_header = df_header[df_header.API.isin(df_raw_production.API)] 

# sorting
df_raw_production = df_raw_production.sort_values(["API", "ReportDate"], ascending = (True, True)) 
df_raw_production = df_raw_production.reset_index(drop=True)


print('raw file: ')
print(Raw_Monthly_Production.shape)
print(Raw_Monthly_Production.API.nunique())
print(Raw_Header.shape)
print(Raw_Header.API.nunique())

print('*'*20)
print('processed file: ')
print(df_raw_production.shape)
print(df_raw_production.API.nunique())
print(df_header.shape)
print(df_header.API.nunique())


raw file: 
(2576773, 11)
27634
(27641, 37)
27641
********************
processed file: 
(2574612, 10)
27608
(27608, 39)
27608


In [18]:
# function 5: make sure monthly production oil, gas and water has no null or negative values

def Fix_Null_Negative(df_raw_production):
    
    df_raw_production['WellOil'].fillna(0,inplace=True)
    df_raw_production['WellGas'].fillna(0,inplace=True)
    df_raw_production['WellWater'].fillna(0,inplace=True)
    
    
    df_raw_production['WellOil'][df_raw_production['WellOil'] < 0] = 0
    df_raw_production['WellGas'][df_raw_production['WellGas'] < 0] = 0
    df_raw_production['WellWater'][df_raw_production['WellWater'] < 0] = 0
    
    return df_raw_production
    

    
df_raw_production = Fix_Null_Negative(df_raw_production)   
    
print(df_raw_production.shape)
print(df_raw_production.API.nunique())

print(df_header.shape)
print(df_header.API.nunique())


(2574612, 10)
27608
(27608, 39)
27608


### Part II: Data Wrangling

In [19]:
# function 1: remove double entry

def Remove_Double_Entry(df_raw_production):
    
    # change the date format to be start of the month
    df_raw_production['ReportDate'] = df_raw_production['ReportDate'].values.astype('datetime64[M]')

    # drop absolutely same records first
    df_trim_production = df_raw_production.drop_duplicates(subset=['API', 'ReportDate','Days',
                                                                'WellOil','WellGas','WellWater'], keep='first')
    
    # drop duplicates where same report date has multiple records
    df_clean_production = df_trim_production.drop_duplicates(subset= ['API','ReportDate'])
    
    # sum the well oil, gas, water records, this is what we agreed on 
    df_clean_production.WellOil = np.asarray(df_trim_production.groupby(['API','ReportDate'])['WellOil'].sum())
    df_clean_production.WellGas = np.asarray(df_trim_production.groupby(['API','ReportDate'])['WellGas'].sum())
    df_clean_production.WellWater = np.asarray(df_trim_production.groupby(['API','ReportDate'])['WellWater'].sum())


    return df_clean_production
    
df_clean_production = Remove_Double_Entry(df_raw_production)
df_header = df_header[df_header.API.isin(df_clean_production['API'])]
df_header.drop_duplicates(subset=['API'],inplace=True)

# check
print('clean_production:')
print(df_clean_production.shape)
print(df_clean_production.API.nunique())
print('*'*20)
print('header:')
print(df_header.shape)
print(df_header.API.nunique())

clean_production:
(2513017, 10)
27608
********************
header:
(27608, 39)
27608


In [20]:
# function 2.1 remove salt water disposial well
# it is defined as cum oil + cum gas == 0 but cum water > 0

df_header['Cum_Oil'] = np.asarray(df_clean_production.groupby('API')['WellOil'].sum())
df_header['Cum_Gas'] = np.asarray(df_clean_production.groupby('API')['WellGas'].sum())
df_header['Cum_Water'] = np.asarray(df_clean_production.groupby('API')['WellWater'].sum())
df_header['Cum_Hydrocarbon'] = df_header['Cum_Oil'] + df_header['Cum_Gas'] 
df_header = df_header[df_header['Cum_Hydrocarbon'] > 0]
df_clean_production = df_clean_production[df_clean_production.API.isin(df_header.API)] 


# check
print('clean_production:')
print(df_clean_production.shape)
print(df_clean_production.API.nunique())
print('*'*20)
print('header:')
print(df_header.shape)
print(df_header.API.nunique())


clean_production:
(2512188, 10)
27600
********************
header:
(27600, 43)
27600


In [21]:
# function 2.2: remove (Oil + Gas + Water <= 0)
# this is aligned with we agreed with Varya
# shut-in defination is now where oil + gas + water <= 0

df_shutin_months = df_clean_production[(df_clean_production['WellOil'] <= 0) &
                                      (df_clean_production['WellGas'] <= 0 ) &
                                      (df_clean_production['WellWater'] <= 0 ) 
                                     ]

to_drop_index = df_shutin_months.index.to_list()

df_clean_production.drop(to_drop_index,inplace=True)

# function 2.1: filter dummy, since we change how we handle shut-in and Days
# this is not need any more
df_header['Filter1'] = 0
df_header['Filter2'] = 0
df_header['Filter3'] = 0
df_header['Filter4'] = 0



print('clean_production:')
print(df_clean_production.shape)
print(df_clean_production.API.nunique())
print('*'*20)
print('header:')
print(df_header.shape)
print(df_header.API.nunique())


clean_production:
(2119910, 10)
27600
********************
header:
(27600, 47)
27600


In [22]:
# function 3: get daily productions

df_final_production = df_clean_production
df_final_production = df_final_production[df_final_production.API.isin(df_header.API)]
df_header = df_header[df_header.API.isin(df_final_production.API)]
df_header.drop_duplicates(subset='API',inplace=True)

df_final_production.drop('MonthNumber', axis=1, inplace=True)
df_final_production['ReportDate'] = pd.to_datetime(df_final_production['ReportDate'])


# Permian origional MonthNumber column is bad, so create new MonthNumber as Rank to fix the bug
df_final_production = df_final_production.drop_duplicates(subset= ['API','ReportDate'])
df_final_production = df_final_production.sort_values(["API", "ReportDate"], ascending = (True, True))
df_final_production['MonthNumber'] = df_final_production.groupby('API')['ReportDate'].rank(ascending=True)
df_final_production["MonthNumber"] = df_final_production["MonthNumber"].astype(np.int64)


df_final_production['Time'] = df_final_production['MonthNumber'] / 12
df_final_production['BOPD_Oil'] = df_final_production['WellOil'] / df_final_production['Days']
df_final_production['BOPD_Gas'] = df_final_production['WellGas'] / df_final_production['Days']
df_final_production['BOPD_Water'] = df_final_production['WellWater'] / df_final_production['Days']



print(df_final_production.shape)
print(df_final_production.API.nunique())
print(df_header.shape)
print(df_header.API.nunique())


(2119910, 14)
27600
(27600, 47)
27600


In [23]:
# function 4: get Jan 2018, 2015, 2012 positions for historical study

### To add the Jan 2018 position for each well to file 1

Jan_2018_Position = []
API_List = df_header.API

for i in API_List:
    API_Oil = df_final_production[df_final_production['API'] == i]
#     print(i)
    s = (API_Oil.loc[API_Oil.index,'ReportDate'] >= np.datetime64('2018-01-01'))
    if not any(s.values) == False:
      
        t = np.where(s.values == True)
        if len(t) > 0:
            Jan_2018_Position.append(t[0][0] + 1)
    
    else:
        Jan_2018_Position.append(-1)

df_header['Jan_2018_Position'] = Jan_2018_Position


### To add the Jan 2015 position for each well to file 1

Jan_2015_Position = []
API_List = df_header.API

for i in API_List:
    API_Oil = df_final_production[df_final_production['API'] == i]
#     print(i)
    s = (API_Oil.loc[API_Oil.index,'ReportDate'] >= np.datetime64('2015-01-01'))
    if not any(s.values) == False:
      
        t = np.where(s.values == True)
        if len(t) > 0:
            Jan_2015_Position.append(t[0][0] + 1)
    
    else:
        Jan_2015_Position.append(-1)

df_header['Jan_2015_Position'] = Jan_2015_Position



### To add the Jan 2012 position for each well to file 1

Jan_2012_Position = []
API_List = df_header.API

for i in API_List:
    API_Oil = df_final_production[df_final_production['API'] == i]
#     print(i)
    s = (API_Oil.loc[API_Oil.index,'ReportDate'] >= np.datetime64('2012-01-01'))
    if not any(s.values) == False:
      
        t = np.where(s.values == True)
        if len(t) > 0:
            Jan_2012_Position.append(t[0][0] + 1)
    
    else:
        Jan_2012_Position.append(-1)

df_header['Jan_2012_Position'] = Jan_2012_Position


In [24]:
# function 5: generate df_QMM file


df_QMM = df_final_production[['API','ReportDate','BOPD_Oil','BOPD_Gas',
                              'BOPD_Water','Days','WellOil','WellGas','WellWater']]
df_QMM['ReportDate'] = pd.to_datetime(df_QMM['ReportDate'])


# Permian origional MonthNumber column is bad, so create new MonthNumber as Rank to fix the bug

df_QMM['Instance'] = df_QMM.groupby('API')['ReportDate'].rank(ascending=True)
df_QMM["Instance"] = df_QMM["Instance"].astype(np.int64)


df_QMM.rename({'BOPD_Oil': 'ActualProdOil', 
                          'BOPD_Gas': 'ActualProdGas',
                          'BOPD_Water': 'ActualProdWater',}, axis=1, inplace=True)

df_QMM.drop_duplicates(subset= ['API','ReportDate'], inplace= True)


In [25]:
# function 6: generate File2, File3 and File4

if BASIN != 'EF' and BASIN != 'MC':

    df_oil = df_QMM.pivot(index='Instance',columns='API',values='ActualProdOil') 
    df_gas = df_QMM.pivot(index='Instance',columns='API',values='ActualProdGas') 
    df_water = df_QMM.pivot(index='Instance',columns='API',values='ActualProdWater') 




    df_oil['Time'] = df_oil.index / 12
    df_oil = df_oil.set_index('Time')
    # df_oil.drop('Instance',axis=1,inplace=True)
    df_oil.to_csv('df_oil_File2.csv')

    df_gas['Time'] = df_gas.index / 12
    df_gas = df_gas.set_index('Time')
    # df_gas.drop('Instance',axis=1,inplace=True)
    df_gas.to_csv('df_gas_File3.csv')

    df_water['Time'] = df_water.index / 12
    df_water = df_water.set_index('Time')
    # df_gas.drop('Instance',axis=1,inplace=True)
    df_water.to_csv('df_water_File4.csv')

In [26]:
# function 7.1:  First/Last Production Date

df_header.drop(['FirstProdDate', 'LastProdDate'], axis=1, inplace=True)
df_QMM = df_QMM[df_QMM.API.isin(df_header.API)]
df_header = df_header[df_header.API.isin(df_QMM.API)]
df_QMM['ReportDate'] = pd.to_datetime(df_QMM['ReportDate'])

df_QMM['ReportDate'] = df_QMM['ReportDate'].values.astype('datetime64[M]')
df_QMM.drop_duplicates(subset= ['API','ReportDate'], inplace= True)


df_FirstProdDate = df_QMM.loc[df_QMM.groupby('API')['ReportDate'].idxmin()]
df_LastProdDate = df_QMM.loc[df_QMM.groupby('API')['ReportDate'].idxmax()]

# df_QMM[df_QMM['API'] == 42389371400000]
# df_FirstProdDate[df_FirstProdDate['API'] == 30015384320000]

df_FirstProdDate_simple = df_FirstProdDate[['API','ReportDate']]
df_FirstProdDate_simple.rename(columns={'ReportDate':'FirstProdDate'}, inplace=True)

df_LastProdDate_simple = df_LastProdDate[['API','ReportDate']]
df_LastProdDate_simple.rename(columns={'ReportDate':'LastProdDate'}, inplace=True)

df_header = pd.merge(df_header, df_FirstProdDate_simple, on='API')
df_header = pd.merge(df_header, df_LastProdDate_simple, on='API')

# df_header['FirstProdDate'] = pd.to_datetime(df_header['FirstProdDate'])
df_header['FirstProdDate'] = df_header['FirstProdDate'].dt.strftime('%m/%d/%Y')
# df_header['LastProdDate'] = pd.to_datetime(df_header['LastProdDate'])
df_header['LastProdDate'] = df_header['LastProdDate'].dt.strftime('%m/%d/%Y')

In [27]:
# function 7.2: calcualte GOR, TotalMonthsHistory

df_header['IP90Oil'] = df_QMM.groupby('API')['WellOil'].apply(lambda x: x.head(3).sum()).values
df_header['IP180Oil'] = df_QMM.groupby('API')['WellOil'].apply(lambda x: x.head(6).sum()).values
df_header['IP360Oil'] = df_QMM.groupby('API')['WellOil'].apply(lambda x: x.head(12).sum()).values
df_header['IP90Gas'] = df_QMM.groupby('API')['WellGas'].apply(lambda x: x.head(3).sum()).values
df_header['IP180Gas'] = df_QMM.groupby('API')['WellGas'].apply(lambda x: x.head(6).sum()).values
df_header['IP360Gas'] = df_QMM.groupby('API')['WellGas'].apply(lambda x: x.head(12).sum()).values
df_header['IP90Water'] = df_QMM.groupby('API')['WellWater'].apply(lambda x: x.head(3).sum()).values
df_header['IP180Water'] = df_QMM.groupby('API')['WellWater'].apply(lambda x: x.head(6).sum()).values
df_header['IP360Water'] = df_QMM.groupby('API')['WellWater'].apply(lambda x: x.head(12).sum()).values
df_header['90DayGOR'] = 1000 * df_header['IP90Gas'] / df_header['IP90Oil']
df_header['180DayGOR'] = 1000 * df_header['IP180Gas'] / df_header['IP180Oil']
df_header['360DayGOR'] = 1000 * df_header['IP360Gas'] / df_header['IP360Oil']
df_header['TotalMonthsHistory'] = df_QMM.groupby('API')['Instance'].max().values
# df_header['Cum_Oil'] = np.asarray(df_QMM.groupby('API')['WellOil'].sum())
# df_header['Cum_Gas'] = np.asarray(df_QMM.groupby('API')['WellGas'].sum())
# df_header['Cum_Water'] = np.asarray(df_QMM.groupby('API')['WellWater'].sum())

df_header['Months_After_Jan_2018'] = df_header['TotalMonthsHistory'] - df_header['Jan_2018_Position'] + 1
df_header['Months_After_Jan_2015'] = df_header['TotalMonthsHistory'] - df_header['Jan_2015_Position'] + 1
df_header['Months_After_Jan_2012'] = df_header['TotalMonthsHistory'] - df_header['Jan_2012_Position'] + 1



In [28]:
# function 8: export all results

df_header['FirstProdDate'] = pd.to_datetime(df_header['FirstProdDate'])
df_header['FirstProdDate'] = df_header['FirstProdDate'].dt.strftime('%m/%d/%Y')
df_header['LastProdDate'] = pd.to_datetime(df_header['LastProdDate'])
df_header['LastProdDate'] = df_header['LastProdDate'].dt.strftime('%m/%d/%Y')
df_QMM['ReportDate'] = pd.to_datetime(df_QMM['ReportDate'])
df_QMM['ReportDate'] = df_QMM['ReportDate'].dt.strftime('%m/%d/%Y')

if BASIN != 'EF' and BASIN != 'MC'  :
    df_header.to_csv('df_header.csv',index=False)
    df_QMM.to_csv('df_QMM.csv',index=False)

if BASIN == 'EF':
    df_header.to_csv('df_header_ALL_EF.csv',index=False)
    df_QMM.to_csv('df_QMM_ALL_EF.csv',index=False)
    
if BASIN == 'MC':
    df_header.to_csv('df_header_ALL_MC.csv',index=False)
    df_QMM.to_csv('df_QMM_ALL_MC.csv',index=False)